# Do feature engineering

In [31]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [19]:
from helpers import createDFFromFileAndSchema

In [13]:
spark=SparkSession.builder.appName('read data through spark').getOrCreate()

### Create dataframes from cleaned data

In [18]:
CLEAN_DATA_FOLDER = './assets/cleanedDFsData/'
SCHEMAS_FOLDER = './schemas/'

# here we should probably update some schemas.
# Because those were made for the initial tables,
# but now we have more columns with the computed values, for example.

author_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}authors.csv', f'{SCHEMAS_FOLDER}author.csv')
research_interests_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}research_interests.csv', f'{SCHEMAS_FOLDER}research_interests.csv')

paper_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}papers.csv', f'{SCHEMAS_FOLDER}paper.csv')
paper_author_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}paper_author.csv', f'{SCHEMAS_FOLDER}paper_authors.csv')
citation_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}citations.csv', f'{SCHEMAS_FOLDER}citations.csv')
affiliation_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}affiliations.csv', f'{SCHEMAS_FOLDER}affiliation.csv')
publication_venue_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}publication_venues.csv', f'{SCHEMAS_FOLDER}publication_venues.csv')

File path: ./assets/cleanedDFsData/authors.csv, schema path: ./schemas/author.csv
Types from schema: [('author_id', 'Integer'), ('citation_count', 'Integer'), ('h_index', 'Integer'), ('name', 'String'), ('paper_count', 'Integer')]
File path: ./assets/cleanedDFsData/research_interests.csv, schema path: ./schemas/research_interests.csv
Types from schema: [('author_id', 'Integer'), ('research_interests', 'String')]
File path: ./assets/cleanedDFsData/papers.csv, schema path: ./schemas/paper.csv
Types from schema: [('paper_id', 'Integer'), ('title', 'String'), ('year', 'Integer')]
File path: ./assets/cleanedDFsData/paper_author.csv, schema path: ./schemas/paper_authors.csv
Types from schema: [('authors', 'String'), ('paper_id', 'Integer')]
File path: ./assets/cleanedDFsData/citations.csv, schema path: ./schemas/citations.csv
Types from schema: [('paper_id', 'Integer'), ('ref_ids', 'String')]
File path: ./assets/cleanedDFsData/affiliations.csv, schema path: ./schemas/affiliation.csv
Types fr

In [29]:
print(research_interests_df["research_interests"])
research_interests_df.printSchema()

Column<'research_interests'>
root
 |-- author_id: integer (nullable = true)
 |-- research_interests: string (nullable = true)



### Create String Indexers and One Hot Encoders for the research_interests_df String columns

In [33]:
# Create a StringIndexer
carr_indexer = StringIndexer(inputCol="research_interests", outputCol="research_interests_index")

# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol="research_interests_index", outputCol="research_interests_fact")

### Create String Indexers and One Hot Encoders for the research_interests_df String columns